In [14]:
import pandas as pd

file_name = "data/parsed_psm/sc_rep5.tsv"
df = pd.read_csv(file_name, sep='\t', index_col=0)
df = df.sort_index()
df = df.set_index(["File Name","scan"]) # Not sure if we need to keep filename in the index. It depends on whether we get unique scans
df = df.rename(columns={"precursor_intenisty": "precursor_intensity"})
df

Matched Ion Series  \
File Name              scan                                                       
D19_15um30cm_SC5-calib 14112  [b2+1, b3+1, b4+1, b5+1, b6+1, b10+1];[y2+1, y...   
                       26732  [y1+1, y2+1, y4+1, y5+1, y6+1, y7+1, y8+1, y9+...   
                       11971  [y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...   
                       13773  [y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...   
                       16447  [y1+1, y2+1, y3+1, y4+1, y5+1, y6+1, y7+1, y8+...   
...                                                                         ...   
                       19022              [y1+1, y4+1, y5+1, y6+1, y7+1, y10+2]   
                       13822           [b3+1];[y6+1, y9+1, y10+1, y11+1, y12+1]   
                       12616              [b2+1];[y2+1, y4+1, y6+1, y7+1, y8+1]   
                       11226             [y1+1, y5+1, y7+1, y9+1, y10+1];[b2+1]   
                       23659              [y1+1, y2+1, y5+1, y6+1, y8+1];[b3+1]   

                                              Matched Ion Mass-To-Charge Ratios  \
File Name              scan                                                       
D19_15um30cm_SC5-calib 14112  [b2+1:200.10295, b3+1:257.12438, b4+1:354.1770...   
                       26732  [y1+1:175.11896, y2+1:274.18715, y4+1:460.2519...   
                       11971  [y1+1:147.11279, y2+1:234.14487, y3+1:348.1879...   
                       13773  [y1+1:147.11271, y2+1:246.18096, y3+1:347.2291...   
                       16447  [y1+1:175.11891, y2+1:246.15591, y3+1:359.2404...   
...                                                                         ...   
                       19022  [y1+1:175.11900, y4+1:514.37129, y5+1:629.3975...   
                       13822  [b3+1:346.11775];[y6+1:731.37946, y9+1:1077.51...   
                       12616  [b2+1:228.13460];[y2+1:304.16150, y4+1:534.215...   
                       11226  [y1+1:147.11298, y5+1:562.28336, y7+1:720.3524...   
                       23659  [y1+1:147.11282, y2+1:260.19690, y5+1:589.3308...   

                                                        Matched Ion Intensities  \
File Name              scan                                                       
D19_15um30cm_SC5-calib 14112  [b2+1:3099, b3+1:1096, b4+1:569, b5+1:545, b6+...   
                       26732  [y1+1:1032, y2+1:1523, y4+1:555, y5+1:3836, y6...   
                       11971  [y1+1:982, y2+1:2498, y3+1:2256, y4+1:5883, y5...   
                       13773  [y1+1:485, y2+1:359, y3+1:813, y4+1:329, y5+1:...   
                       16447  [y1+1:1467, y2+1:1597, y3+1:2155, y4+1:752, y5...   
...                                                                         ...   
                       19022  [y1+1:829, y4+1:877, y5+1:1856, y6+1:2065, y7+...   
                       13822  [b3+1:520];[y6+1:267, y9+1:1096, y10+1:1334, y...   
                       12616  [b2+1:272];[y2+1:818, y4+1:555, y6+1:685, y7+1...   
                       11226  [y1+1:263, y5+1:330, y7+1:2250, y9+1:3238, y10...   
                       23659  [y1+1:497, y2+1:284, y5+1:494, y6+1:467, y8+1:...   

                              Matched Ion Counts    QValue  \
File Name              scan                                  
D19_15um30cm_SC5-calib 14112                  28  0.000000   
                       26732                  26  0.000000   
                       11971                  25  0.000000   
                       13773                  25  0.000000   
                       16447                  24  0.000000   
...                                          ...       ...   
                       19022                   6  0.009988   
                       13822                   6  0.009988   
                       12616                   6  0.009988   
                       11226                   6  0.009988   
                       23659                   6  0.009988   

              

In [15]:
df_series = df.drop(columns=["Matched Ion Intensities","Matched Ion Mass-To-Charge Ratios"])
df_intensities = df[["Matched Ion Intensities"]].copy()
df_mz = df[["Matched Ion Mass-To-Charge Ratios"]].copy()

# Deal with getting the Ion Name, Ion Charge, Ion Number (with respect to the peptide)
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(";")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.split(", ")
df_series = df_series.explode("Matched Ion Series")
df_series["Matched Ion Series"] = df_series["Matched Ion Series"].str.strip("[]")
df_series = df_series.rename(columns={"Matched Ion Series": "Ion Name"})
df_series[["Ion Type", "Ion Number", "Ion Charge"]] = df_series["Ion Name"].str.extract(r"([abcxyz])(\d{1,3})([+-]\d)") # Tried to handle every possible case here, but this may need to be amended
df_series = df_series.astype({"Ion Charge": int, "Ion Number": int})
df_series = df_series.set_index("Ion Name", append=True)

# Deal with getting the Ion Intensity
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(";")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(", ")
df_intensities = df_intensities.explode("Matched Ion Intensities")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.strip("[]")
df_intensities["Matched Ion Intensities"] = df_intensities["Matched Ion Intensities"].str.split(":")
df_intensities = pd.DataFrame(df_intensities["Matched Ion Intensities"].to_list(), columns=["Ion Name", "Intensity"], index=df_intensities.index)
df_intensities = df_intensities.set_index("Ion Name", append=True)
df_intensities = df_intensities.astype({"Intensity": float})

# Deal with getting the Ion m/z
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(";")
df_mz = df_mz.explode("Matched Ion Mass-To-Charge Ratios")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(", ")
df_mz = df_mz.explode("Matched Ion Mass-To-Charge Ratios")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.strip("[]")
df_mz["Matched Ion Mass-To-Charge Ratios"] = df_mz["Matched Ion Mass-To-Charge Ratios"].str.split(":")
df_mz = pd.DataFrame(df_mz["Matched Ion Mass-To-Charge Ratios"].to_list(), columns=["Ion Name", "Mass-To-Charge Ratio"], index=df_mz.index)
df_mz = df_mz.set_index("Ion Name", append=True)
df_mz = df_mz.astype({"Mass-To-Charge Ratio": float})

df_clean = df_series.join(df_intensities, how="inner")
df_clean = df_clean.join(df_mz, how="inner")

df_clean = df_clean[["Ion Type", "Ion Number", "Ion Charge", "Mass-To-Charge Ratio", "Intensity", "Base Peak Intensity", "mz_array", "intensity_array", "precursor_intensity"]]
df_clean

Ion Type  Ion Number  Ion Charge  \
File Name              scan  Ion Name                                    
D19_15um30cm_SC5-calib 14112 b2+1            b           2           1   
                             b3+1            b           3           1   
                             b4+1            b           4           1   
                             b5+1            b           5           1   
                             b6+1            b           6           1   
...                                        ...         ...         ...   
                       23659 y2+1            y           2           1   
                             y5+1            y           5           1   
                             y6+1            y           6           1   
                             y8+1            y           8           1   
                             b3+1            b           3           1   

                                       Mass-To-Charge Ratio  Intensity  \
File Name              scan  Ion Name                                    
D19_15um30cm_SC5-calib 14112 b2+1                 200.10295     3099.0   
                             b3+1                 257.12438     1096.0   
                             b4+1                 354.17704      569.0   
                             b5+1                 425.21402      545.0   
                             b6+1                 496.25121      890.0   
...                                                     ...        ...   
                       23659 y2+1                 260.19690      284.0   
                             y5+1                 589.33085      494.0   
                             y6+1                 688.39952      467.0   
                             y8+1                 930.53643      566.0   
                             b3+1                 333.08642     1101.0   

                                       Base Peak Intensity  \
File Name              scan  Ion Name                        
D19_15um30cm_SC5-calib 14112 b2+1              3432.428955   
                             b3+1              3432.428955   
                             b4+1              3432.428955   
                             b5+1              3432.428955   
                             b6+1              3432.428955   
...                                                    ...   
                       23659 y2+1              1418.214111   
                             y5+1              1418.214111   
                             y6+1              1418.214111   
                             y8+1              1418.214111   
                             b3+1              1418.214111   

                                                                                mz_array  \
File Name              scan  Ion Name                                                      
D19_15um30cm_SC5-calib 14112 b2+1      [115.0872573852539, 127.08719635009766, 129.10...   
                             b3+1      [115.0872573852539, 127.08719635009766, 129.10...   
                             b4+1      [115.0872573852539, 127.08719635009766, 129.10...   
                             b5+1      [115.0872573852539, 127.08719635009766, 129.10...   
                             b6+1      [115.0872573852539, 127.08719635009766, 129.10...   
...                                                                                  ...   
                       23659 y2+1      [110.07189178466797, 113.07160949707031, 115.0...   
                             y5+1      [110.07189178466797, 113.07160949707031, 115.0...   
                             y6+1      [110.07189178466797, 113.07160949707031, 115.0...   
                             y8+1      [110.07189178466797, 113.07160949707031, 115.0...   
                             b3+1      [110.07189178466797, 113.07160949707031, 115.0...   

                                                                         intensity_array  \
File Name       